In [1]:

!pip list

Package           Version
----------------- -------
asttokens         2.4.0
backcall          0.2.0
colorama          0.4.6
comm              0.1.4
debugpy           1.7.0
decorator         5.1.1
exceptiongroup    1.1.3
executing         1.2.0
ipykernel         6.25.2
ipython           8.15.0
jedi              0.19.0
jupyter_client    8.3.1
jupyter_core      5.3.1
matplotlib-inline 0.1.6
nest-asyncio      1.5.7
packaging         23.1
parso             0.8.3
pickleshare       0.7.5
pip               23.2.1
platformdirs      3.10.0
prompt-toolkit    3.0.39
psutil            5.9.5
pure-eval         0.2.2
Pygments          2.16.1
python-dateutil   2.8.2
pywin32           306
pyzmq             25.1.1
setuptools        58.1.0
six               1.16.0
stack-data        0.6.2
tornado           6.3.3
traitlets         5.9.0
wcwidth           0.2.6


In [3]:
!pip install opencv-contrib-python

  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/05/33/5a6436146bda09c69decc456cfb54f41d52fbcf558fe91e6df7bdde6cce0/opencv_contrib_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for numpy>=1.21.2 from https://files.pythonhosted.org/packages/b7/db/4d37359e2c9cf8bf071c08b8a6f7374648a5ab2e76e2e22e3b808f81d507/numpy-1.25.2-cp310-cp310-win_amd64.whl.metadata
  Using cached numpy-1.25.2-cp310-cp310-win_amd64.whl.metadata (5.7 kB)
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/44.8 MB 8.9 MB/s eta 0:00:05
   ---------------------------------------- 0.4/44.8 MB 5.4 MB/s eta 0:00:09
   ---------------------------------------- 0.5/44.8 MB 3.6 MB/s eta 0:00:13
   ---------------------------------------- 0.5/44.8 MB 3.1 MB/s eta 0:00:15
    ---------

   ---------- ----------------------------- 12.1/44.8 MB 396.5 kB/s eta 0:01:23
   ---------- ----------------------------- 12.1/44.8 MB 396.5 kB/s eta 0:01:23
   ---------- ----------------------------- 12.2/44.8 MB 393.0 kB/s eta 0:01:24
   ---------- ----------------------------- 12.3/44.8 MB 393.5 kB/s eta 0:01:23
   ----------- ---------------------------- 12.3/44.8 MB 393.5 kB/s eta 0:01:23
   ----------- ---------------------------- 12.3/44.8 MB 393.2 kB/s eta 0:01:23
   ----------- ---------------------------- 12.3/44.8 MB 392.7 kB/s eta 0:01:23
   ----------- ---------------------------- 12.4/44.8 MB 392.0 kB/s eta 0:01:23
   ----------- ---------------------------- 12.4/44.8 MB 390.9 kB/s eta 0:01:23
   ----------- ---------------------------- 12.4/44.8 MB 390.2 kB/s eta 0:01:23
   ----------- ---------------------------- 12.5/44.8 MB 389.7 kB/s eta 0:01:23
   ----------- ---------------------------- 12.5/44.8 MB 389.0 kB/s eta 0:01:23
   ----------- -------------------------

In [8]:
import os
import numpy as np
import cv2

In [59]:
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    cv2.imshow('Image', image)
    cv2.waitKey(0)

    # Background Removal (Assuming the background is relatively uniform)
    # Example: Use color-based segmentation to separate the leaf from the background
#     lower_bound = np.array([0, 40, 0], dtype="uint8")
#     upper_bound = np.array([100, 255, 100], dtype="uint8")
#     mask = cv2.inRange(image, lower_bound, upper_bound)
#     result = cv2.bitwise_and(image, image, mask=mask)
    
#     cv2.imshow('Bg remove',result)
#     cv2.waitKey(0)

    # Noise Reduction (Gaussian Blur)
#     result = cv2.GaussianBlur(image, (5, 5), 0)
    
#     cv2.imshow('Bg remove',result)
#     cv2.waitKey(0)

    # Color Normalization (Convert to LAB color space and equalize channels)
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)
    l_eq = cv2.equalizeHist(l)
    normalized_lab = cv2.merge([l_eq, a, b])
    result = cv2.cvtColor(normalized_lab, cv2.COLOR_LAB2BGR)
    
    cv2.imshow('Bg remove',result)
    cv2.waitKey(0)

    # Contrast Enhancement (Histogram Equalization)
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    result = clahe.apply(gray)

    # Edge Detection (Canny)
    edges = cv2.Canny(result, 30, 150)

    # Morphological Operations (Dilation and Erosion)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    edges = cv2.erode(edges, kernel, iterations=1)

    return edges  # Preprocessed image with highlighted edges

In [60]:
def calculate_severity(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)

    # Calculate total leaf area (excluding background)
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total_leaf_area = sum(cv2.contourArea(contour) for contour in contours)
    
    cv2.imshow('Pre', preprocessed_image)
    cv2.waitKey(0)

    # Calculate affected area by summing up contour areas
    affected_area = sum(cv2.contourArea(contour) for contour in contours)

    # Calculate severity score (difference-based approach)
    severity_score = (total_leaf_area - affected_area) / total_leaf_area

    return severity_score

In [61]:
# Example usage:
image_path = './images/apple_rust.jpeg'
severity = calculate_severity(image_path)
print(f"Severity Score: {severity}")

Severity Score: 0.0


In [63]:
cv2.destroyAllWindows()